<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/models/2DModels/2DCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")
import zipfile


Mounted at /content/drive


In [ ]:
zip_ref = zipfile.ZipFile('drive/Shareddrives/Datasets SEFAI/fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

# **Dataset Loading**

In [ ]:
train_dir = "train" # Directory containing the training data
test_dir = "test"  # Directory containing the validation data

# Define the transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),

])

test_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = ImageFolder(train_dir, transform=train_transform)
test_dataset = ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# **Rete convolutiva**

In [ ]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, 1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;


In [ ]:

class CNN(nn.Module):
    def __init__(self):

        super(CNN, self).__init__()


        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.dropout2 = nn.Dropout(0.25)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)

        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)

        self.dropout3 = nn.Dropout(0.25)

        self.fc1 = nn.Linear(256 * 6 * 6, 256)

        self.bn7 = nn.BatchNorm1d(256)

        self.dropout4 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(256, 7)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.ReLU()(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.ReLU()(x)

        x = self.pool(x)

        x = self.dropout1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = nn.ReLU()(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = nn.ReLU()(x)

        x = self.pool(x)
        x = self.dropout2(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = nn.ReLU()(x)

        x = self.conv6(x)
        x = self.bn6(x)
        x = nn.ReLU()(x)

        x = self.pool(x)
        x = self.dropout3(x)

        x = x.view(-1, 256 * 6 * 6)

        x = self.fc1(x)
        x = self.bn7(x)
        x = nn.ReLU()(x)
        x = self.dropout4(x)
        x = self.fc2(x)
        return x



In [ ]:
model = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)

In [ ]:

# Training loop
best_accuracy = 0.0

for epoch in range(50):
    model.train()
    running_loss = 0.0
    total_correct = 0
    seen = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, acc = accuracy(outputs, labels)
        total_correct += acc
        seen +=labels.shape[0]
        running_loss += loss.item()

    train_accuracy = total_correct / seen
    train_loss = running_loss / len(train_loader)

    model.eval()
    total_correct = 0
    seen = 0
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, acc = accuracy(outputs, labels)
            total_correct += acc
            seen +=labels.shape[0]

    test_accuracy = total_correct / seen
    test_loss = running_loss / len(test_loader)
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), 'model_weights.pth')

    print(f"Epoch {epoch+1}/{50}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


Epoch 1/50, Train Loss: 1.7688, Train Accuracy: 0.2911, Test Accuracy: 0.3895, Test Loss: 1.5819
Epoch 2/50, Train Loss: 1.5638, Train Accuracy: 0.3932, Test Accuracy: 0.4448, Test Loss: 1.4690
Epoch 3/50, Train Loss: 1.4494, Train Accuracy: 0.4431, Test Accuracy: 0.4862, Test Loss: 1.3410
Epoch 4/50, Train Loss: 1.3784, Train Accuracy: 0.4721, Test Accuracy: 0.4925, Test Loss: 1.3102
Epoch 5/50, Train Loss: 1.3289, Train Accuracy: 0.4955, Test Accuracy: 0.5192, Test Loss: 1.2572
Epoch 6/50, Train Loss: 1.2868, Train Accuracy: 0.5102, Test Accuracy: 0.5191, Test Loss: 1.2321
Epoch 7/50, Train Loss: 1.2553, Train Accuracy: 0.5213, Test Accuracy: 0.5166, Test Loss: 1.2343
Epoch 8/50, Train Loss: 1.2275, Train Accuracy: 0.5347, Test Accuracy: 0.5486, Test Loss: 1.1764
Epoch 9/50, Train Loss: 1.1988, Train Accuracy: 0.5444, Test Accuracy: 0.5541, Test Loss: 1.1546
Epoch 10/50, Train Loss: 1.1817, Train Accuracy: 0.5498, Test Accuracy: 0.5554, Test Loss: 1.1463
Epoch 11/50, Train Loss: 1.15